##Frame2seq: structure-conditioned masked language modeling for protein sequence design

Official notebook for [Frame2seq](https://doi.org/10.1101/2023.12.15.571823): Structure-conditioned masked language models for protein sequence design generalize beyond the native sequence space.

In [ ]:
#@title Install dependencies
%%capture
!pip install frame2seq

import os
from google.colab import files


def get_pdb(pdb_code=""):
# code credit: https://github.com/sokrypton/ColabDesign notebooks
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  elif os.path.isfile(pdb_code):
    return pdb_code
  elif len(pdb_code) == 4:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"
  else:
    os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/AF-{pdb_code}-F1-model_v3.pdb")
    return f"AF-{pdb_code}-F1-model_v3.pdb"

In [ ]:
#@title Frame2seq Runner
%%capture
from frame2seq import Frame2seqRunner


runner = Frame2seqRunner()

In [ ]:
#@title Design sequences with Frame2seq (sampled sequences are saved to .fasta files)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# USER OPTIONS
#@markdown #### Input options for fixed-backbone design
pdb='2FRA' #@param {type:"string"}
#@markdown - leave blank to get an upload prompt
chain_id = "A" #@param {type:"string"}
temperature = 1.0 #@param {type:"raw"}
num_samples = 10 #@param {type:"raw"}
save_neg_pll = True #@param {type:"boolean"}
#@markdown - saves per-position negative pseudo-log-likelihoods to csv
verbose = True #@param {type:"boolean"}

pdb_path = get_pdb(pdb)

# run Frame2seq
runner.design(pdb_path, chain_id, temperature, num_samples, save_neg_pll, verbose)